In [1]:
import librosa
import matplotlib.pyplot as plt
import librosa.display
import os
import numpy as np
from tensorflow.keras import layers
from IPython.display import Audio
import wave
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

2024-02-21 21:56:09.487828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 21:56:09.488098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 21:56:09.489762: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 21:56:09.498770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-21 21:56:10.558939: W tensorflow/compiler/tf2

In [2]:
df = pd.read_csv("../data_processing/sep28k-mfcc.csv")

In [3]:
df = df[df['NaturalPause'] == 0]
df = df[df['Interjection'] == 0]
df = df[df['Prolongation'] == 0]
df = df[df['WordRep'] == 0]
df = df[df['Block'] == 0]
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,29,30,31,32,33,34,35,36,37,38
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,-3.249218,-3.218731,0.360520,-0.115838,1.723229,-1.460772,-1.430723,-1.429669,-0.178196,-3.046811
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,-2.057481,-3.225782,-1.561315,-3.723942,-2.249673,-3.513439,-2.097951,-1.940040,0.020210,-0.556461
6,HeStutters,0,6,37251200,37299200,0,0,0,0,0,...,-4.985494,-6.181957,-2.396787,-6.544606,-2.319257,-3.688410,-0.221778,-1.690031,0.664374,-0.303436
9,HeStutters,0,9,41417440,41465440,0,0,0,0,0,...,-4.798577,-6.274436,-4.889234,-5.795599,-4.400376,-4.176473,-0.957432,-3.210292,0.577023,1.042720
11,HeStutters,0,11,42928800,42976800,0,0,0,0,0,...,-3.798326,-6.705790,-1.544637,-6.356187,-0.987506,-3.107216,-1.422836,-3.615824,-1.171095,-1.937332


In [4]:
df = df.reset_index()
df.head()

,index,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,...,29,30,31,32,33,34,35,36,37,38
0,2,HeStutters,0,2,34809760,34857760,0,0,0,0,...,-3.249218,-3.218731,0.360520,-0.115838,1.723229,-1.460772,-1.430723,-1.429669,-0.178196,-3.046811
1,4,HeStutters,0,4,35721920,35769920,0,0,0,0,...,-2.057481,-3.225782,-1.561315,-3.723942,-2.249673,-3.513439,-2.097951,-1.940040,0.020210,-0.556461
2,6,HeStutters,0,6,37251200,37299200,0,0,0,0,...,-4.985494,-6.181957,-2.396787,-6.544606,-2.319257,-3.688410,-0.221778,-1.690031,0.664374,-0.303436
3,9,HeStutters,0,9,41417440,41465440,0,0,0,0,...,-4.798577,-6.274436,-4.889234,-5.795599,-4.400376,-4.176473,-0.957432,-3.210292,0.577023,1.042720
4,11,HeStutters,0,11,42928800,42976800,0,0,0,0,...,-3.798326,-6.705790,-1.544637,-6.356187,-0.987506,-3.107216,-1.422836,-3.615824,-1.171095,-1.937332


In [5]:
df = df.drop(columns=['index'])
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,29,30,31,32,33,34,35,36,37,38
0,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,-3.249218,-3.218731,0.360520,-0.115838,1.723229,-1.460772,-1.430723,-1.429669,-0.178196,-3.046811
1,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,-2.057481,-3.225782,-1.561315,-3.723942,-2.249673,-3.513439,-2.097951,-1.940040,0.020210,-0.556461
2,HeStutters,0,6,37251200,37299200,0,0,0,0,0,...,-4.985494,-6.181957,-2.396787,-6.544606,-2.319257,-3.688410,-0.221778,-1.690031,0.664374,-0.303436
3,HeStutters,0,9,41417440,41465440,0,0,0,0,0,...,-4.798577,-6.274436,-4.889234,-5.795599,-4.400376,-4.176473,-0.957432,-3.210292,0.577023,1.042720
4,HeStutters,0,11,42928800,42976800,0,0,0,0,0,...,-3.798326,-6.705790,-1.544637,-6.356187,-0.987506,-3.107216,-1.422836,-3.615824,-1.171095,-1.937332


In [6]:
df.to_csv("soundrep.csv",index=False)

In [7]:
import os

def list_files(directory):
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# Replace 'directory_path' with the path to your directory
directory_path = '/home/alien/Git/DATA/mel_spects_soundrep'
# names_list = list_files(directory_path)
names_list = pd.read_csv("soundrep.csv")['Name'].values.tolist()
# full_names_list = ["/home/alien/Git/DATA/mfcc_images/" + img + ".jpg" for img in names_list]
full_names_list = []
for img in names_list:
    corresponding_sound = df.loc[df['Name'] == img, 'SoundRep'].values[0]
    if corresponding_sound == 0:
        full_names_list.append("/home/alien/Git/DATA/mel_spects_soundrep/" + img + "_fluent.jpg")
    if corresponding_sound >= 1:
        full_names_list.append("/home/alien/Git/DATA/mel_spects_soundrep/" + img + "_stutter.jpg")

print(full_names_list[-2])

/home/alien/Git/DATA/mel_spects_soundrep/WomenWhoStutter_109_33_fluent.jpg


In [8]:
def load_all(imagefile_list):
    data = []
    labels = []

    for imagefile in imagefile_list:
        print(imagefile)
        image = cv2.imread(imagefile)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        data.append(image)

        if "fluent" in imagefile:
            labels.append(0)
        else:
            labels.append(1)

    labels = np.array(labels)
    data = np.array(data)

    return data, labels

In [9]:
X, y = load_all(full_names_list)

/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_2_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_4_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_6_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_9_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_11_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_12_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_15_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_16_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_20_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_22_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_24_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_26_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_29_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_32_fluent.jpg
/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_

In [10]:
print(X[0])
print(y[0])

[[[ 68  33  60]
  [ 96  28  68]
  [ 89  27  67]
  ...
  [  0   0   4]
  [  0   0   4]
  [  0   0   2]]

 [[114  57  88]
  [166  57 117]
  [157  55 112]
  ...
  [  0   0   4]
  [  0   0   4]
  [  0   0   2]]

 [[140  60  94]
  [190  55 118]
  [198  65 126]
  ...
  [  0   0   4]
  [  0   0   4]
  [  0   0   2]]

 ...

 [[126  65 103]
  [194  56 123]
  [189  69 128]
  ...
  [ 29  13  72]
  [ 18   5  59]
  [ 21   6  57]]

 [[ 90  48 102]
  [145  31 116]
  [141  39 122]
  ...
  [ 26  18  68]
  [ 24  17  63]
  [ 27  19  61]]

 [[ 86  33  88]
  [122  36 120]
  [134  36 131]
  ...
  [ 23  14  63]
  [ 27  18  70]
  [ 26  14  66]]]
0


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.1, random_state=12)

In [12]:
print("x_train shape:", X_train.shape)
print("x_test shape:", X_test.shape)
print('y_train shape:', y_train.shape)
print("y_test shape:", y_test.shape)

x_train shape: (1747, 224, 224, 3)
x_test shape: (195, 224, 224, 3)
y_train shape: (1747,)
y_test shape: (195,)


In [13]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

1747 195 1747 195


In [14]:
from collections import Counter
print(Counter(y_train))
print(Counter(y_test))

Counter({0: 1530, 1: 217})
Counter({0: 168, 1: 27})


In [15]:
print(X_train)

[[[[  1   2   0]
   [  2   4   0]
   [  1   3   0]
   ...
   [  0   0   4]
   [  0   0   4]
   [  0   0   2]]

  [[  1   0   3]
   [  1   0   4]
   [  1   0   3]
   ...
   [  0   0   4]
   [  0   0   4]
   [  0   0   1]]

  [[  6   0  13]
   [  8   2  16]
   [  6   1  17]
   ...
   [  1   1   2]
   [  0   0   2]
   [  0   0   0]]

  ...

  [[  1   1   0]
   [  2   1   5]
   [  1   0   6]
   ...
   [  0   0   1]
   [  0   0   1]
   [  0   0   2]]

  [[  1   1   1]
   [  2   1   6]
   [  1   0   8]
   ...
   [  0   0   7]
   [  0   0   5]
   [  0   0   4]]

  [[  1   0   7]
   [  2   1   7]
   [  1   0   6]
   ...
   [  0   0   3]
   [  0   0   3]
   [  0   0   2]]]


 [[[ 13  14  38]
   [ 24  11  57]
   [ 54  19  81]
   ...
   [  0   0   4]
   [  0   0   4]
   [  0   0   2]]

  [[ 25  18  52]
   [ 47  20  83]
   [ 94  42 125]
   ...
   [  0   0   4]
   [  0   0   4]
   [  0   0   2]]

  [[ 28  17  57]
   [ 47  14  87]
   [ 90  32 128]
   ...
   [  0   0   4]
   [  0   0   4]
   [  0   0

In [16]:
from PIL import Image
IMAGE_DIR = "/home/alien/Git/DATA/mel_spects/"

def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height

image_path = '/home/alien/Git/DATA/mel_spects_soundrep/HeStutters_0_0_fluent.jpg'  # Change this to the path of your image file
width, height = get_image_dimensions(image_path)
print("Image width:", width)
print("Image height:", height)

Image width: 610
Image height: 450


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, MultiHeadAttention
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input, AveragePooling2D, Attention, Reshape, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.metrics import classification_report, confusion_matrix

In [18]:
# def build_model(input_shape=(224, 224, 3)):
#     base_model = VGG19(weights='imagenet', include_top=True, input_tensor=Input(shape=input_shape))
#     # Get the output of the 'fc2' layer in VGG16
#     a = base_model.get_layer('fc2').output

#     # Add a Dense layer with 13 neurons
#     dense_layer = Dense(13, activation='relu')(a)

#     # Add a final output layer with sigmoid activation
#     output_layer = Dense(1, activation='sigmoid')(dense_layer)

#     # Define the model with VGG16 base and the added layers
#     model = Model(inputs=base_model.input, outputs=output_layer)

#     # Freeze the weights of the VGG16 layers
#     for layer in base_model.layers:
#         layer.trainable = False

#     return model

def build_model(input_shape=(224, 224, 3), use_float16=False):
    inputs = Input(shape=input_shape)

    # Convert to lower precision if specified
    if use_float16:
        x = tf.keras.layers.Lambda(lambda t: tf.cast(t, tf.float16))(inputs)
    else:
        x = inputs

    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)

    x = TimeDistributed(Flatten())(x)  # Flatten along the time dimension
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    # x = Flatten()(x)
    x = Attention()([x, x])  # Self-attention mechanism

    # Flatten and fully connected layers
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)  # Adding dropout with a rate of 0.5
    
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model


In [19]:
  # base_model = VGG19(weights='imagenet', include_top=True,
  #                   input_tensor=Input(shape=(224, 224, 3)))
  # base_model.summary()

In [20]:
vgg_model = build_model()
vgg_model.summary()

2024-02-21 21:56:21.162063: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-21 21:56:21.189008: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-21 21:56:21.189459: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

2024-02-21 21:56:21.668883: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 222, 222, 32)         896       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)         0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         128       ['max_pooling2d[0][0]']   

In [21]:
vgg_model.compile(
  optimizer=Adam(lr=0.01),
  loss='binary_crossentropy',
  metrics=['accuracy']
)

In [22]:
# batch_size = 16
# history = vgg_model.fit(
#     X_train,
#     y_train,
#     batch_size=batch_size,
#     epochs=35,
#     callbacks=[
#         ReduceLROnPlateau(
#             monitor = 'accuracy',
#             factor = 0.2,
#             patience = 5,
#             verbose = 1,
#             min_lr = 0.0001
#         ),
#         EarlyStopping(
#             monitor = 'accuracy',
#             patience = 10,
#             verbose = 1,
#             restore_best_weights = True
#         )
#     ]
# )

batch_size = 32
history = vgg_model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=35,
    callbacks=[
        ReduceLROnPlateau(
            monitor = 'accuracy',
            factor = 0.2,
            patience = 5,
            verbose = 1,
            min_lr = 0.0001
        ),
    ]
)

Epoch 1/35


W0000 00:00:1708570586.292901  469440 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 3080" frequency: 1710 num_cores: 68 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 5242880 shared_memory_size_per_multiprocessor: 102400 memory_size: 7266238464 bandwidth: 760080000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2024-02-21 21:56:26.656318: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-21 21:56:26.786465: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-21 21:56:28.688774: I external/local_xla/xla/service/serv

55/55 [==============================] - 11s 39ms/step - loss: 0.4245 - accuracy: 0.8626 - lr: 0.0010
Epoch 2/35
55/55 [==============================] - 1s 21ms/step - loss: 0.3575 - accuracy: 0.8786 - lr: 0.0010
Epoch 3/35
55/55 [==============================] - 1s 20ms/step - loss: 0.3200 - accuracy: 0.8895 - lr: 0.0010
Epoch 4/35
55/55 [==============================] - 1s 20ms/step - loss: 0.2506 - accuracy: 0.9090 - lr: 0.0010
Epoch 5/35
55/55 [==============================] - 1s 20ms/step - loss: 0.1782 - accuracy: 0.9365 - lr: 0.0010
Epoch 6/35
55/55 [==============================] - 1s 20ms/step - loss: 0.1250 - accuracy: 0.9525 - lr: 0.0010
Epoch 7/35
55/55 [==============================] - 1s 20ms/step - loss: 0.0821 - accuracy: 0.9685 - lr: 0.0010
Epoch 8/35
55/55 [==============================] - 1s 20ms/step - loss: 0.0780 - accuracy: 0.9725 - lr: 0.0010
Epoch 9/35
55/55 [==============================] - 1s 20ms/step - loss: 0.0660 - accuracy: 0.9765 - lr: 0.0010
Ep

In [23]:
# predictions
vgg_pred = vgg_model.predict(X_test, batch_size=1)

vgg_pred = np.round(vgg_pred)

  # model evaluation
confusion = confusion_matrix(y_test, vgg_pred)
print(classification_report(y_test, vgg_pred))
print(confusion)

W0000 00:00:1708570632.919697  469440 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 3080" frequency: 1710 num_cores: 68 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 5242880 shared_memory_size_per_multiprocessor: 102400 memory_size: 7266238464 bandwidth: 760080000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


195/195 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           0       0.86      0.98      0.92       168
           1       0.20      0.04      0.06        27

    accuracy                           0.85       195
   macro avg       0.53      0.51      0.49       195
weighted avg       0.77      0.85      0.80       195

[[164   4]
 [ 26   1]]


In [24]:
vgg_model.save('./vgg19_model_v2', overwrite=True)

INFO:tensorflow:Assets written to: ./vgg19_model_v2/assets


INFO:tensorflow:Assets written to: ./vgg19_model_v2/assets
